In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode,  plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
import numpy as np
import math
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA
import warnings
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import warnings
warnings.filterwarnings('ignore')
params = {'legend.fontsize': 'xx-large',
          'figure.figsize': (15, 10),
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'xx-large',
         'ytick.labelsize':'xx-large'}

In [31]:
# functions to fill the missing  values using interpolation
#'Monthly_Avg_Tkt_Rev','Monthly_Avg_Tkt_Rev_PnS','Monthly Parts & Supplies MTD % to Quota','Monthly SMA MTD % to Quota','Monthly Store Account MTD % to Quota','Monthly WC MTD % to Quota','Monthly Mktg Adopt'
# test=pd.read_csv("X1.csv")
Sales = pd.read_excel('transforedstoremetrics.xlsx', sheetname='2014')
# test1=pd.read_excel('transforedstoremetrics.xlsx', sheetname='2013')
def interpolate_data(column_name):
    Sales[column_name].interpolate(method='linear', inplace=True,limit_direction='both')

In [32]:
# Date Conversion function
# Plant_Master=pd.read_csv("Plant_Master.csv")
Sales['Month']=Sales['TS_ID']
Sales['Year']=Sales['Year']
Sales['Date'] = pd.to_datetime(Sales[['Year', 'Month']].assign(Day=1))
Sales['Pandas_Datestamp'] = pd.to_datetime(Sales['Date'], infer_datetime_format=True)
Sales['Year'] = pd.DatetimeIndex(Sales['Date']).year
Sales['TS_ID'] = pd.DatetimeIndex(Sales['Date']).month
Sales.sort_values(by='Pandas_Datestamp')
Plants = Sales.Store_ID.unique()

In [34]:
# interpolate_data('Monthly Avg Tkt Rev')

Sales_col=['Monthly VOC','Monthly Avg Tkt Rev PnS','Monthly Parts & Supplies MTD % to Quota','Monthly SMA MTD % to Quota','Monthly Store Account MTD % to Quota','Monthly WC MTD % to Quota','Monthly Mktg Adopt']
for i in Sales_col:
    interpolate_data(i)
Sales.to_csv("transforedstoremetrics2014.csv")  

In [35]:
#FUNCTION FOR DIFFERENT PLANT
def diff_plant(plant,predictor):
  
    
    
    Plant_1=pd.DataFrame(Sales[Sales['Store_ID']== plant]) #EXTRACTING A SPECIFIC Plant/Store
#     Sales[predictor] = pd.to_numeric(Sales[predictor], errors='coerce')
#     cols_to_norm = [predictor] #SCALING
#     Plant_1[cols_to_norm] = Plant_1[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min())) #SCALING
#     Plant_1.sort_values(by='Pandas_Datestamp')


#     SEPARATING AS PER YEAR
#     Plant_1_2013=Plant_1[Plant_1['Year']==2013]
#     Plant_1_2013['Month'] = Plant_1_2013['Pandas_Datestamp'].apply(lambda x: x.strftime('%Y-%m-01')) #FIRST DAY OF EVERY MONTH
#     Plant_1_2013=pd.DataFrame(Plant_1_2013.groupby('Month', as_index=False)[predictor].mean())
#     Plant_1_2013.sort_values(by='Month')

    Plant_1_2014=Plant_1[Plant_1['Year']==2014]
    Plant_1_2014['Month'] = Plant_1_2014['Pandas_Datestamp'].apply(lambda x: x.strftime('%Y-%m-01')) #FIRST DAY OF EVERY MONTH
    Plant_1_2014=pd.DataFrame(Plant_1_2014.groupby('Month', as_index=False)[predictor].mean())
    Plant_1_2014.sort_values(by='Month')

#CONCATENATION

    Plant_1_ALLYEARS = pd.concat([Plant_1_2014]).reset_index(drop=True) #, axis=1
    Plant_1_ALLYEARS.index = Plant_1_ALLYEARS['Month']
    Plant_1_ALLYEARS.drop(columns='Month')
    Plant_1_ALLYEARS= Plant_1_ALLYEARS.drop(columns='Month')
    Plant_1_ALLYEARS.reset_index(inplace=True)
    Plant_1_ALLYEARS['Month'] = pd.to_datetime(Plant_1_ALLYEARS['Month'])
    Plant_1_ALLYEARS = Plant_1_ALLYEARS.set_index('Month')
    #ROLLING AVERGAGE FORMULA - TRIAL WITH MOVING WINDOW
#     Plant_1_ALLYEARS['MA_3']= Plant_1_ALLYEARS[predictor].rolling(3).mean()
#     Plant_1_ALLYEARS['MA_3_std']= Plant_1_ALLYEARS[predictor].rolling(3).std() #QUATERLY
    Plant_1_ALLYEARS['Plant'] = plant
    return Plant_1_ALLYEARS

In [37]:
# to generate moving average values for 2015 for all the coulmns
import warnings
warnings.filterwarnings('ignore')
def moving_average(data,j,store_id):
    values=data.values[:,0]
#     print(values)
    average=[]
    for i in range(1,13):
#         print(len)
        avg=round((values[len(values)-1]+values[len(values)-2])/2,6)
        values=np.append(values,round(avg,6))
        average.append([('2015-'+str(i)+'-'+'01'),avg,store_id])
#     df=pd.DataFrame(average,columns=['Date',j,Store_id])
    return average
# moving_average(diff_plant('A168','Monthly WC MTD % to Quota'),'Monthly WC MTD % to Quota','A168')
# df[i]=pd.DataFrame(data, columns=['Date',j])
# diff_plant('A168','Monthly WC MTD % to Quota')

In [38]:
# df=pd.DatFrame
p=0
q=0

data_final=[]

# 'Monthly VOC','Monthly Phonecalls','Monthly Mktg Adopt','Monthly Avg Tkt YoY Growth','Monthly Avg Tkt Rev PnS','Monthly Parts & Supplies MTD % to Quota','Monthly Store Account MTD % to Quota','Monthly SMA MTD % to Quota'
Sales_col=['Monthly VOC','Monthly Phonecalls','Monthly Mktg Adopt','Monthly Avg Tkt YoY Growth','Monthly Avg Tkt Rev PnS','Monthly Parts & Supplies MTD % to Quota','Monthly Store Account MTD % to Quota','Monthly SMA MTD % to Quota']    
for k,j in enumerate(Sales_col):
    data=[]
    for i in range(len(Plants)):
        t=moving_average(diff_plant(Plants[i],j),j,Plants[i])
        s=pd.DataFrame(t,columns=['Date',j,'Store_id'])
        data.append(s)
    data_final.append(pd.concat(data))
#     print(result)
#     data_final.append(result)

In [39]:
# melt for each catogories and concat, but because duplicted values add cumcount for unique triples in MultiIndex
df1 = data_final[0].set_index(['Store_id','Date', data_final[0].groupby(['Store_id','Date']).cumcount()])
df2 = data_final[1].set_index(['Store_id','Date', data_final[1].groupby(['Store_id','Date']).cumcount()])
df3 = data_final[2].set_index(['Store_id','Date', data_final[2].groupby(['Store_id','Date']).cumcount()])
df4 = data_final[3].set_index(['Store_id','Date', data_final[3].groupby(['Store_id','Date']).cumcount()])
df5 = data_final[4].set_index(['Store_id','Date', data_final[4].groupby(['Store_id','Date']).cumcount()])
df6 = data_final[5].set_index(['Store_id','Date', data_final[5].groupby(['Store_id','Date']).cumcount()])
df7 = data_final[6].set_index(['Store_id','Date', data_final[6].groupby(['Store_id','Date']).cumcount()])
df8 = data_final[7].set_index(['Store_id','Date', data_final[7].groupby(['Store_id','Date']).cumcount()])
# df9 = data_final[8].set_index(['Store_id','Date', data_final[8].groupby(['Store_id','Date']).cumcount()])
result=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8],axis=1).sort_index(level=2).reset_index(level=2, drop=True).reset_index()
# data_final[1]